# Spark SQL and Analytics on Iguazio

1. [Overview](#Overview)
2. [Set Up](#Set-Up)
2. [Inititate a Spark Session](#Inititate-a-Spark-Session)
3. [Load Data into Spark DataFrame](#Load-Data)
  1. [Load Data from S3](#Load-Data-from-S3)
  2. [Load Data from External Table](#Load-Data-from-External-Table)
  3. [Load Data from UnStructure File](#Load-Data-from-UnStructure-File)
  4. [Overwrite Table Schema](#Overwrite-Table-Schema)
4. [Spark SQL](#Spark-SQL)
  1. [Spark SQL on an Object](#Spark-SQL-on-an-Object)
  2. [Spark SQL on a Table](#Spark-SQL-on-a-Table)
  3. [Spark SQL on Iguazio](#Spark-SQL-on-Iguazio)
  4. [Spark SQL Join](#Spark-SQL-Join)
  5. [Spark SQL on a Parque File](#Spark-SQL-on-a-Parque-File)
  6. [Spark SQL on a Partition Table](#Spark-SQL-on-a-Partition-Table)
5. [Conditional Update on Iguazio](#Conditional-Update-on-Iguazio)
6. [Clean Up](#Clean-Up)


# Overview

**Spark SQL** is Apache Spark's module for working with structured data.  Spark SQL lets you query structured data inside Spark programs, using either SQL or a familiar DataFrame API.  DataFrames and SQL provide a common way to access a variety of data sources. <br>

In this notebook, let's walk through how Spark SQL and DataFrames access objects, tables, unstructured data persisted on [**Iguazio Data Flatform**](https://www.iguazio.com/). <br>

Iguazio drivers for Spark implement the data-source API and allow `predicate push down` (the queries pass to Iguazio datastore that only returns the relevant data), this allow accelerated and high-speed access from Spark to data stored in **Iguazio Data Flatform/Fabric**. <br>

For more details read [Spark SQL and DataFrames documentation](https://spark.apache.org/docs/2.3.1/sql-programming-guide.html)<br>

# Set Up

Before preparing the data, there are some initial steps required for setting up environmentn variables. <br>

* Iguazio environmentn variables
* AWS Credintial and Bucket

In [1]:
import os

# Iguazio env
V3IO_USER = os.getenv('V3IO_USERNAME')
%env V3IO_HOME = /v3io/users/iguazio

# AWS Credintial and Bucket
%env AWS_USER = '<your_aws_user>'
%env AWS_PASSWORD = '<your_aws_password>'
%env BUCKET = '<your_s3_bucket_name_here>'
%env ACCESSKEY = '<your_accesskey.'
%env SECRETKEY = '<your_secretkey>'

# printenv

env: V3IO_HOME=/v3io/users/iguazio
env: AWS_USER='<your_aws_user>'
env: AWS_PASSWORD='<your_aws_password>'
env: BUCKET='<your_s3_bucket_name_here>'
env: ACCESSKEY='<your_accesskey.'
env: SECRETKEY='<your_secretkey>'


# Inititate a Spark Session

For getting started, you can start with Spark default configurations.

Other than using the default Spark configurations, you may need modify some Spark properties to suite for your applications.  Based on the nature of datasets and data models, the way how to access the data, as well as the hardware resources, apply different configurations may lead to differnt performance.  You will go through some experiements in the late [sections](#Experiments-of-SQL-performance-on-a-Partition-KV-Store).  <br>

* Some primary Spark properties are: <br>
    `spark.driver.cores` <br>
    `spark.driver.memory` <br>
    `spark.executor.cores` <br>
    `spark.executor.memory` <br>
    `spark.cores.max` <br>

    `spark.python.profile` <br>
    `spark.pyspark.python` <br>

* Set up the follow Spark properties for accessing AWS S3 bucket: <br>
    `spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem` <br>
    `spark.hadoop.fs.s3a.access.key=ACCESSKEY` <br>
    `spark.hadoop.fs.s3a.secret.key=SECRETKEY` <br>
    `spark.hadoop.fs.s3a.fast.upload=true` <br>

For more details read [Spark Properties](https://spark.apache.org/docs/2.3.1/configuration.html#dynamically-loading-spark-properties) <br>

For further performance services and support, contact our [Professional Services](https://www.iguazio.com/support/). <br>

## Spark Configuration

In [2]:
%%time

from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, col

# Initiate a Spark Session
spark = SparkSession.builder.appName("Spark Session with Default Configurations").getOrCreate()

# Get the default configurations
spark.sparkContext._conf.getAll()

conf = spark.sparkContext._conf

print('\n\nCurrent Spark Configuration is : \n')

print('spark.app.name = ', conf.get("spark.app.name"))
print('spark.driver.cores = ', conf.get("spark.driver.cores"))
print('spark.driver.memory = ', conf.get("spark.driver.memory"))
print('spark.executor.cores = ', conf.get("spark.executor.cores"))
print('spark.executor.memory = ', conf.get("spark.executor.memory"))
print('spark.cores.max = ', conf.get("spark.cores.max"))

print('spark.python.profile = ', conf.get("spark.python.profile"))
print('spark.pyspark.python = ', conf.get("spark.pyspark.python"))
print('spark.hadoop.fs.s3a.impl = ', conf.get("spark.hadoop.fs.s3a.impl"))

# Update the default configurations
# Here is an example of using one m5.2xlarge (8cpu, 32G) as application node.
'''
conf = spark.sparkContext._conf\
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
'''
conf = spark.sparkContext._conf\
    .setAll([('spark.app.name', 'Spark SQL for Analytics'), \
             # ('spark.driver.cores', '2'), \                # Only in cluster mode.
             ('spark.driver.memory','2g'), 
             ('spark.executor.cores', '2'), \
             ('spark.executor.memory', '4g'), \
             ('spark.cores.max', '3'), \
             ('spark.python.profile', 'true'), \
             ('spark.pyspark.python', 'true'), \
             ("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")])

# Stop the current Spark Session
spark.sparkContext.stop()

# Create a Spark Session with new configurations
spark = SparkSession.builder.config(conf=conf).appName("Spark SQL for Analytics - ipynb").getOrCreate()

print('\n\nChange Spark Configuration to : \n')

print('spark.app.name = ', conf.get("spark.app.name"))
print('spark.driver.cores = ', conf.get("spark.driver.cores"))
print('spark.driver.memory = ', conf.get("spark.driver.memory"))
print('spark.executor.cores = ', conf.get("spark.executor.cores"))
print('spark.executor.memory = ', conf.get("spark.executor.memory"))
print('spark.cores.max = ', conf.get("spark.cores.max"))

print('spark.python.profile = ', conf.get("spark.python.profile"))
print('spark.pyspark.python = ', conf.get("spark.pyspark.python"))
print('spark.hadoop.fs.s3a.impl = ', conf.get("spark.hadoop.fs.s3a.impl"))

print('\n')



Current Spark Configuration is : 

spark.app.name =  Spark Session with Default Configurations
spark.driver.cores =  None
spark.driver.memory =  None
spark.executor.cores =  2
spark.executor.memory =  None
spark.cores.max =  None
spark.python.profile =  None
spark.pyspark.python =  None
spark.hadoop.fs.s3a.impl =  None


Change Spark Configuration to : 

spark.app.name =  Spark SQL for Analytics
spark.driver.cores =  None
spark.driver.memory =  2g
spark.executor.cores =  2
spark.executor.memory =  4g
spark.cores.max =  3
spark.python.profile =  true
spark.pyspark.python =  true
spark.hadoop.fs.s3a.impl =  org.apache.hadoop.fs.s3a.S3AFileSystem


CPU times: user 173 ms, sys: 34.3 ms, total: 207 ms
Wall time: 6.06 s


# Load Data

The Spark Data Sources API supports a pluggable mechanism for integration with structured data-sources.  It is a unified API designed to support two major operations:

1. Loading structured data from an external data source into Spark
2. Storing structured data from Spark into an external data source.

## Load Data from S3

Load a file from S3 to Spark DataFrame <br>
File URL of the form `s3a://bucket/path/to/file` <br>

## Or Copy a file from AWS S3 to Iguazio
Alternative, you can copy the data to Iguazio application node first. <br>
Here, you copy a csv file from an AWS S3 to Iguazio. <br>

In [3]:
!curl -L "iguazio-sample-data.s3.amazonaws.com/2018-03-26_BINS_XETR08.csv" > ${V3IO_HOME}/stocks.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  861k  100  861k    0     0  1116k      0 --:--:-- --:--:-- --:--:-- 1115k


In [ ]:
#!ls -alt /v3io/users/iguazio/stocks.csv

### Move data into an Iguazio Data Container <br>
Let's create a directory with name *stock*, then move data into Iguazio data container.<br>

In [4]:
!hadoop fs -mkdir v3io://users/iguazio/stock/
!hadoop fs -copyFromLocal /v3io/users/iguazio/stocks.csv v3io://users/iguazio/stock/

19/02/28 08:08:57 INFO slf_4j.Slf4jLogger: Slf4jLogger started
mkdir: `v3io://users/iguazio/stock': File exists
19/02/28 08:09:01 INFO slf_4j.Slf4jLogger: Slf4jLogger started
copyFromLocal: `v3io://users/iguazio/stock/stocks.csv': File exists


### List files in Iguazion Data Container

In [6]:
!hadoop fs -ls v3io://users/iguazio/stock/

19/02/28 08:10:22 INFO slf_4j.Slf4jLogger: Slf4jLogger started
Found 1 items
-rw-r--r--   1 50 nogroup     882055 2019-02-28 06:18 v3io://users/iguazio/stock/stocks.csv


### Create a Spark DataFrame, load a Iguazio file <br>

Here, use Infer Schema to create a dataframe that infers the input schema automatically from data. <br>
Also, you can specify a schema instead. <br>

`schema = StructType(fields)` <br>
`df = spark.read...option ("Schema", schema)....` <br>

In [3]:
%%time

df = spark.read\
    .format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("v3io://users/iguazio/stock/stocks.csv")

CPU times: user 3.35 ms, sys: 676 µs, total: 4.02 ms
Wall time: 10.1 s


### Print out Schema

In [5]:
df.printSchema()

root
 |-- ISIN: string (nullable = true)
 |-- Mnemonic: string (nullable = true)
 |-- SecurityDesc: string (nullable = true)
 |-- SecurityType: string (nullable = true)
 |-- Currency: string (nullable = true)
 |-- SecurityID: integer (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Time: string (nullable = true)
 |-- StartPrice: double (nullable = true)
 |-- MaxPrice: double (nullable = true)
 |-- MinPrice: double (nullable = true)
 |-- EndPrice: double (nullable = true)
 |-- TradedVolume: integer (nullable = true)
 |-- NumberOfTrades: integer (nullable = true)



### List Columns

In [6]:
df.columns

['ISIN',
 'Mnemonic',
 'SecurityDesc',
 'SecurityType',
 'Currency',
 'SecurityID',
 'Date',
 'Time',
 'StartPrice',
 'MaxPrice',
 'MinPrice',
 'EndPrice',
 'TradedVolume',
 'NumberOfTrades']

## Load Data from External Table

### Use MySQL as External Data Source
Reading from MySQL as a bulk operation using pandas dataframe.

**NOTE** If this notebook runs in AWS Cloud:
AWS S3 provides **eventual consistency**.  Therefore, it takes time for users using the persisted data and softwar package.

In [42]:
# install MySQL lib
!pip install pymysql 

    100% |################################| 51kB 19.8MB/s 


In [7]:
import os
import pymysql
import pandas as pd 

conn = pymysql.connect(
    host=os.getenv('DB_HOST','mysql-rfam-public.ebi.ac.uk'),
    port=int(4497),
    user=os.getenv('DB_USER','rfamro'),
    passwd=os.getenv('DB_PASSWORD',''),
    db=os.getenv('DB_NAME','Rfam'),
    charset='utf8mb4')

dfMySQL = pd.read_sql_query("select rfam_acc,rfam_id,auto_wiki,description,author,seed_source FROM family",
    conn) 

dfMySQL.tail(3)

,rfam_acc,rfam_id,auto_wiki,description,author,seed_source
3013,RF03113,Poribacteria-1,2702,Poribacteria-1 RNA,Weinberg Z,Weinberg Z
3014,RF03114,RT-1,2572,RT-1 RNA,Weinberg Z,Weinberg Z
3015,RF03115,KDPG-aldolase,2703,KDPG-aldolase RNA,Weinberg Z,Weinberg Z


### Use Amazon Redshift as External Data Source

The `spark-redshift` library is a data source API for [Amazon Redshift](https://aws.amazon.com/redshift/). <br>
For this example let's create a S3 bucket `redshift-spark`. <br>

In [ ]:
tmpS3Dir = "s3n://redshift-spark/tmp/"

#### Redshift Environment Setup

In [ ]:
redshiftDBName = '<your_redshift_DB_name>'
redshiftTableName = '<your_redshift_Table_name>'
redshiftUserId = '<your_redshift_User_ID>'
redshiftPassword = '<your_redshift_Password>'
redshifturl = '<your_redshift_URL>'
jdbcURL = s"jdbc:redshift://$redshifturl/$redshiftDBName?user=$redshiftUserId&password=$redshiftPassword"


#### Load Redshift Table into Spark DF
The `.format("com.databricks.spark.redshift")` line tells the Data Sources API that we are using the `spark-redshift` package. <br>
Enable `spark-redshift` to use the `tmpS3Dir` temporary location in S3 to store temporary files generated by `spark-redshift`. <br>

In [ ]:
dfRDSHFT = spark.read
	.format("com.databricks.spark.redshift")
	.option("url",jdbcURL )
	.option("tempdir", tmpS3Dir)
	.option("dbtable", redshiftTableName)
	.load()

#### Print Schema and Show a few Records
`spark-redshift` automatically reads the schema from the Redshift table and maps its types back to Spark SQL's types.

In [ ]:
dfRDSHFT.printSchema()
dfRDSHFT.show(3)

#### Persist Redshift table data into Iguazio KV store

In [ ]:
dfRDSHFT = spark.write\
    .format("io.iguaz.v3io.spark.sql.kv")\
    .mode("append")\
    .option("key", key)\
    .option("sorting-key", sorting-key)\
    .option("allow-overwrite-schema", "true")\
    .save(os.path.join(os.getenv('V3IO_HOME'))+'/rdshfttable/')

## Load Data from UnStructure File

## Overwrite Table Schema

The following example creates a table named mytable with AttrA and AttrB attributes of type string and an AttrC attribute of type long, and then overwrites the table schema to change the type of AttrC to double:

In [8]:
dfOWSchema = spark.createDataFrame([
    ("a", "z", 123),
    ("b", "y", 456)
], ["AttrA", "AttrB", "AttrC"])
dfOWSchema.write.format("io.iguaz.v3io.spark.sql.kv") \
    .mode("overwrite") \
    .option("key", "AttrA") \
    .save(os.path.join(os.getenv('V3IO_HOME'))+'/mytable/')
    
dfOWSchema = spark.createDataFrame([
    ("c", "x", 32.12),
    ("d", "v", 45.2)
], ["AttrA", "AttrB", "AttrC"])
dfOWSchema.write.format("io.iguaz.v3io.spark.sql.kv") \
    .mode("append") \
    .option("key", "AttrA") \
    .option("allow-overwrite-schema", "true") \
    .save(os.path.join(os.getenv('V3IO_HOME'))+'/mytable/')

# Spark SQL

Now, let's run some Spark SQL for analyze the stock dataset that was loaded to df DataFrame. <br>
The fisrt a few SQLs list a few lines of selected columns in the dataset, as well as get some statistics of numerical columns. <br>

## Spark SQL on an Object

In [9]:
df.select("ISIN", "Mnemonic", "SecurityDesc", "SecurityType").show(3)

+------------+--------+--------------------+------------+
|        ISIN|Mnemonic|        SecurityDesc|SecurityType|
+------------+--------+--------------------+------------+
|CH0038389992|    BBZA|BB BIOTECH NAM.  ...|Common stock|
|CH0038863350|    NESR|NESTLE NAM.      ...|Common stock|
|LU0378438732|    C001|COMSTAGE-DAX UCIT...|         ETF|
+------------+--------+--------------------+------------+
only showing top 3 rows



### Retreive first a few rows

In [10]:
df.select("ISIN", "Mnemonic", "SecurityDesc").head(3)

[Row(ISIN='CH0038389992', Mnemonic='BBZA', SecurityDesc='BB BIOTECH NAM.   SF 0,20'),
 Row(ISIN='CH0038863350', Mnemonic='NESR', SecurityDesc='NESTLE NAM.        SF-,10'),
 Row(ISIN='LU0378438732', Mnemonic='C001', SecurityDesc='COMSTAGE-DAX UCITS ETF I')]

### Summary and Descriptive Statistics
The function **describe** returns a DataFrame containing information such as number of non-null entries (count), mean, standard deviation, and minimum and maximum value for each numerical column.

In [11]:
df.describe("TradedVolume").show()

+-------+------------------+
|summary|      TradedVolume|
+-------+------------------+
|  count|              7401|
|   mean|3035.7574652074045|
| stddev|18191.489026530675|
|    min|                 0|
|    max|            839200|
+-------+------------------+



## Spark SQL on a Table

### Register as a Table for futher Analytics

In [12]:
df.createOrReplaceTempView("stock")

### Select a few Columns and Only Print out a few lines

In [13]:
q = spark.sql("SELECT ISIN, SecurityDesc, SecurityID FROM stock limit 3").show()

+------------+--------------------+----------+
|        ISIN|        SecurityDesc|SecurityID|
+------------+--------------------+----------+
|CH0038389992|BB BIOTECH NAM.  ...|   2504244|
|CH0038863350|NESTLE NAM.      ...|   2504245|
|LU0378438732|COMSTAGE-DAX UCIT...|   2504271|
+------------+--------------------+----------+



### Analyze Data to Find a Column or a few Columns for the Unique Key

In [14]:
q1 = spark.sql("SELECT COUNT(ISIN) FROM stock").show()
q2 = spark.sql("SELECT COUNT(DISTINCT(ISIN)) FROM stock").show()

+-----------+
|count(ISIN)|
+-----------+
|       7401|
+-----------+

+--------------------+
|count(DISTINCT ISIN)|
+--------------------+
|                 737|
+--------------------+



In [15]:
q4 = spark.sql("SELECT COUNT(SecurityID) FROM stock").show()
q5 = spark.sql("SELECT COUNT(DISTINCT(SecurityID)) FROM stock").show()

+-----------------+
|count(SecurityID)|
+-----------------+
|             7401|
+-----------------+

+--------------------------+
|count(DISTINCT SecurityID)|
+--------------------------+
|                       737|
+--------------------------+



### Combination of ISIN, Date, Time can be the Unqiue Key

In [16]:
q6 = spark.sql("SELECT COUNT(DISTINCT(ISIN, Date, Time)) FROM stock").show()

+----------------------------------------------------------------+
|count(DISTINCT named_struct(ISIN, ISIN, Date, Date, Time, Time))|
+----------------------------------------------------------------+
|                                                            7401|
+----------------------------------------------------------------+



### Concatenate Date and Time columns

In [17]:
df.select(concat(col("Date"), col("Time"))).head(2)

[Row(concat(Date, Time)='2018-03-26 00:00:0008:00'),
 Row(concat(Date, Time)='2018-03-26 00:00:0008:00')]

In [18]:
df.withColumn("datetime", concat(col("Date"), col("Time")))\
    .select("Date", "Time", "datetime").head(3)

[Row(Date=datetime.datetime(2018, 3, 26, 0, 0), Time='08:00', datetime='2018-03-26 00:00:0008:00'),
 Row(Date=datetime.datetime(2018, 3, 26, 0, 0), Time='08:00', datetime='2018-03-26 00:00:0008:00'),
 Row(Date=datetime.datetime(2018, 3, 26, 0, 0), Time='08:00', datetime='2018-03-26 00:00:0008:00')]

### Register another Table with a Unique Key

In [19]:
df.withColumn("datetime", concat(df["Date"], df["Time"])).createOrReplaceTempView("stock_UUID")

### Verify the Key is Unique

In [20]:
q7 = spark.sql("SELECT COUNT(DISTINCT(ISIN, datetime)) FROM stock_UUID").show()

+------------------------------------------------------------+
|count(DISTINCT named_struct(ISIN, ISIN, datetime, datetime))|
+------------------------------------------------------------+
|                                                        7401|
+------------------------------------------------------------+



### Get Distinct values on a column

In [21]:
%%time
q8 = spark.sql("SELECT COUNT(DISTINCT(datetime)) FROM stock_UUID").show()

+------------------------+
|count(DISTINCT datetime)|
+------------------------+
|                      60|
+------------------------+

CPU times: user 2.57 ms, sys: 0 ns, total: 2.57 ms
Wall time: 1.27 s


Results show that **All data in this stock dataset is of the same date.**

In [22]:
%time
q9 = spark.sql("SELECT COUNT(DISTINCT(Time)) FROM stock_UUID").show()

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.63 µs
+--------------------+
|count(DISTINCT Time)|
+--------------------+
|                  60|
+--------------------+



## Spark SQL on Iguazio

### Persist Data from Spark DF to an Iguazio KV store
Specify the format to `io.iguaz.v3io.spark.sql.kv` <br>
Specify the path to the NoSQL table that is associated with the DataFrame as a fully qualified path of the format `v3io://Data_Container/Path` — where Data_Container is the name of the table’s data container, and Path is the path to the data within the specified container. <br>
Specify the key and sorting-key (optional). <br>
* KEY = key.sorting-key <br>

**NOTE**: The KEY must be unique. <br>

In [6]:
%%time

# UUID = key.sorting-key
# key: ISIN
# sorting-key : Date + Time
df.withColumn("datetime", concat(df["Date"], df["Time"]))\
    .write\
    .format("io.iguaz.v3io.spark.sql.kv")\
    .mode("append")\
    .option("key", "ISIN")\
    .option("sorting-key", "datetime")\
    .option("allow-overwrite-schema", "true")\
    .save('v3io://users/iguazio/stock/stocks_kv/')

CPU times: user 5.69 ms, sys: 0 ns, total: 5.69 ms
Wall time: 3.78 s


### Load a KV store from Iguazion to Spark DF

In [23]:
df2 = spark.read.format("io.iguaz.v3io.spark.sql.kv").load('v3io://users/iguazio/stock/stocks_kv/')

In [24]:
%%time
df2.select("ISIN", "datetime").head(1)

CPU times: user 5.05 ms, sys: 3.07 ms, total: 8.12 ms
Wall time: 222 ms


[Row(ISIN='DE0006969603', datetime='2018-03-26 00:00:0008:16')]

In [25]:
df2.createOrReplaceTempView("stock_kv")

In [26]:
%%time

q10 = spark.sql("SELECT ISIN, SUM(TradedVolume) FROM stock_kv GROUP BY ISIN").show(5)

+------------+-----------------+
|        ISIN|sum(TradedVolume)|
+------------+-----------------+
|LU0488317024|               35|
|DE0005570808|             3350|
|FI0009000681|            51473|
|DE000A1K0375|             2000|
|DE000A0H08M3|             5394|
+------------+-----------------+
only showing top 5 rows

CPU times: user 1.58 ms, sys: 1.23 ms, total: 2.82 ms
Wall time: 2.95 s


### Persist Data to another Iguazio KV store with Partition

Partions are firstly by `Date`, and then by `Time`. <br>

In [7]:
%%time

# UUID = key.sorting-key
# key: ISIN
# partition : Date, time
df.write\
    .format("io.iguaz.v3io.spark.sql.kv")\
    .mode("append")\
    .option("key", "ISIN")\
    .option("partition", "Date, Time")\
    .save('v3io://users/iguazio/stock/stocks_kv_partition/')

CPU times: user 3.63 ms, sys: 0 ns, total: 3.63 ms
Wall time: 8.32 s


### Load another KV store from Iguazio to Spark DF

In [27]:
df3 = spark.read.format("io.iguaz.v3io.spark.sql.kv").load('v3io://users/iguazio/stock/stocks_kv_partition/')

In [28]:
%%time

df3.select("ISIN", "Date", "Time").show(3)

+------------+-------------------+-----+
|        ISIN|               Date| Time|
+------------+-------------------+-----+
|DE000A1X3XX4|2018-03-26 00:00:00|08:00|
|DE000TLX1005|2018-03-26 00:00:00|08:00|
|DE0005232805|2018-03-26 00:00:00|08:00|
+------------+-------------------+-----+
only showing top 3 rows

CPU times: user 1.92 ms, sys: 2.33 ms, total: 4.25 ms
Wall time: 2.36 s


### Experiments of SQL performance on a Partition KV Store

In [7]:
df3.createOrReplaceTempView("stock_kv_partintion")

**NOTE** This partition data model does not well support for the following query.

In [ ]:
%%time
# 1*m5.2xlarge: Spark 4 executor, 2core and 1G perexecutor, never return results
# 1*m5.2xlarge: Spark 2 executor, 2core and 4G per executor, never return results
#%debug
#q11 = spark.sql("SELECT ISIN, SUM(TradedVolume) FROM stock_kv_partintion GROUP BY ISIN").show(5)
#q11 = spark.sql("SELECT ISIN, SUM(TradedVolume) FROM stock_kv_partintion GROUP BY ISIN").explain

#### More Experiments
The following experiments are about running the same query with different [Spark configuration](#Spark-Configuration)

In [7]:
%%time
# 1*m5.2xlarge: Spark 4 executor, 2core and 1G per executor

q12 = spark.sql("SELECT Date, SUM(TradedVolume) FROM stock_kv_partintion GROUP BY Date").show()

+-------------------+-----------------+
|               Date|sum(TradedVolume)|
+-------------------+-----------------+
|2018-03-26 00:00:00|         22467641|
+-------------------+-----------------+

CPU times: user 19.8 ms, sys: 2 ms, total: 21.8 ms
Wall time: 2min 4s


In [7]:
%%time
# 1*m5.2xlarge: Spark 2 executor, 2core and 4G per executor,

q12 = spark.sql("SELECT Date, SUM(TradedVolume) FROM stock_kv_partintion GROUP BY Date").show()

+-------------------+-----------------+
|               Date|sum(TradedVolume)|
+-------------------+-----------------+
|2018-03-26 00:00:00|         22467641|
+-------------------+-----------------+

CPU times: user 18.1 ms, sys: 4.03 ms, total: 22.1 ms
Wall time: 2min 9s


In [12]:
%%time
# 1*m5.2xlarge: Spark 4 executor, 1core and 4G per executor,

q12 = spark.sql("SELECT Date, SUM(TradedVolume) FROM stock_kv_partintion GROUP BY Date").show()

+-------------------+-----------------+
|               Date|sum(TradedVolume)|
+-------------------+-----------------+
|2018-03-26 00:00:00|         22467641|
+-------------------+-----------------+

CPU times: user 16.8 ms, sys: 4.96 ms, total: 21.7 ms
Wall time: 2min 9s


In [16]:
%%time
# 1*m5.2xlarge: Spark 3 executor, 1core and 8G per executor,

q12 = spark.sql("SELECT Date, SUM(TradedVolume) FROM stock_kv_partintion GROUP BY Date").show()

+-------------------+-----------------+
|               Date|sum(TradedVolume)|
+-------------------+-----------------+
|2018-03-26 00:00:00|         22467641|
+-------------------+-----------------+

CPU times: user 17.9 ms, sys: 4.51 ms, total: 22.4 ms
Wall time: 2min 9s


In [20]:
%%time
# 1*m5.2xlarge: Spark 6 executor, 1core and 4G per executor,

q12 = spark.sql("SELECT Date, SUM(TradedVolume) FROM stock_kv_partintion GROUP BY Date").show()

+-------------------+-----------------+
|               Date|sum(TradedVolume)|
+-------------------+-----------------+
|2018-03-26 00:00:00|         22467641|
+-------------------+-----------------+

CPU times: user 21.5 ms, sys: 2.86 ms, total: 24.4 ms
Wall time: 2min 28s


In [4]:
%%time
# 1*m5.2xlarge: Spark 3 executor, 1core and 8G per executor,

q12 = spark.sql("SELECT Date, SUM(TradedVolume) FROM stock_kv_partintion GROUP BY Date").show()

+-------------------+-----------------+
|               Date|sum(TradedVolume)|
+-------------------+-----------------+
|2018-03-26 00:00:00|         22467641|
+-------------------+-----------------+

CPU times: user 15.4 ms, sys: 7.37 ms, total: 22.7 ms
Wall time: 2min 10s


In [8]:
%%time
# 1*m5.2xlarge: Spark 1 executor, 1core and 20G per executor,

q12 = spark.sql("SELECT Date, SUM(TradedVolume) FROM stock_kv_partintion GROUP BY Date").show()

+-------------------+-----------------+
|               Date|sum(TradedVolume)|
+-------------------+-----------------+
|2018-03-26 00:00:00|         22467641|
+-------------------+-----------------+

CPU times: user 13.2 ms, sys: 8.05 ms, total: 21.2 ms
Wall time: 2min 9s


## Spark SQL Join

In [15]:
dfL = spark.createDataFrame([("2504271", "LU0378438732")], ["SecurityID", "ISIN"])
dfR = spark.createDataFrame([("2504271", "JOIN in Spark SQL")], ["SecurityID", "SQL Query"])

In [16]:
dfJoin = dfL.join(dfR, dfL.SecurityID == dfR.SecurityID).show()

+----------+------------+----------+-----------------+
|SecurityID|        ISIN|SecurityID|        SQL Query|
+----------+------------+----------+-----------------+
|   2504271|LU0378438732|   2504271|JOIN in Spark SQL|
+----------+------------+----------+-----------------+



In [27]:
dfL.createOrReplaceTempView("t1")
dfR.createOrReplaceTempView("t2")

In [28]:
qJoin = spark.sql("SELECT * FROM t1, t2 where t1.SecurityID=t2.SecurityID").show()

+----------+------------+----------+-----------------+
|SecurityID|        ISIN|SecurityID|        SQL Query|
+----------+------------+----------+-----------------+
|   2504271|LU0378438732|   2504271|JOIN in Spark SQL|
+----------+------------+----------+-----------------+



## Spark SQL on a Parque File
### Persist Data into Iguazio Data Container in Parque format
Use the same stock dataset to store in Parque format.

In [29]:
%%time

df.write\
    .mode("overwrite")\
    .parquet("v3io://users/iguazio/stock/stocks_parq/")

CPU times: user 1.98 ms, sys: 322 µs, total: 2.3 ms
Wall time: 1.25 s


In [31]:
dfPARQ = spark.read.parquet("v3io://users/iguazio/stock/stocks_parq/")

In [33]:
dfPARQ.select("ISIN", "Date").head()

Row(ISIN='CH0038389992', Date=datetime.datetime(2018, 3, 26, 0, 0))

## Spark SQL on a Partition Table

### Create a partition table

This examples creates a partitioned “weather” table  The option("partition", "year, month, day") write option partitions the table by the year, month, and day item attributes. As demonstrated in the following image, if you browse the container in the dashboard after running the example, you’ll see that the weather directory has year=<value>/month=<value>/day=<value> partition directories that match the written items. If you select any of the nested day partition directories, you can see the written items and their attributes. For example, the first item (with attribute values 2016, 3, 25, 6, 16, 0.00, 55) is saved to a 20163256 file in a weather/year=2016/month=3/day=25 partition directory.

In [35]:
table_path = os.path.join(os.getenv('V3IO_HOME')+'/examples/weather/')

df = spark.createDataFrame([
    (2016,  3, 25, 17, 18, 0.2, 62),
    (2016,  7, 24,  7, 19, 0.0, 52),
    (2016, 12, 24,  9, 10, 0.1, 47),
    (2017,  5,  7, 14, 21, 0.0, 70),
    (2017, 11,  1, 10, 15, 0.0, 34),
    (2017, 12, 12, 16, 12, 0.0, 47),
    (2017, 12, 24, 17, 11, 1.0, 50),
    (2018,  1, 18, 17, 10, 2.0, 45),
    (2018,  5, 20, 21, 20, 0.0, 59),
    (2018, 11,  1, 11, 11, 0.1, 65)
], ["year", "month", "day", "hour", "degrees_cel", "rain_ml", "humidity_per"])

df_with_key = df.withColumn(
    "time", concat(df["year"], df["month"], df["day"], df["hour"]))

df_with_key.write.format("io.iguaz.v3io.spark.sql.kv") \
    .mode("overwrite") \
    .option("key", "time") \
    .option("partition", "year, month, day, hour") \
    .save(table_path)

### Reading from partition table
Following is the output of the example’s show commands for each read.  The filtered results are gathered by scanning only the partition directories that match the filter criteria.


#### A Full Table Scan

In [36]:
readDF = spark.read.format("io.iguaz.v3io.spark.sql.kv").load(table_path)
readDF.show()

+----+-----+---+----+-----------+-------+------------+----------+
|year|month|day|hour|degrees_cel|rain_ml|humidity_per|      time|
+----+-----+---+----+-----------+-------+------------+----------+
|2016|   12| 24|   9|         10|    0.1|          47| 201612249|
|2016|    3| 25|  17|         18|    0.2|          62| 201632517|
|2016|    7| 24|   7|         19|    0.0|          52|  20167247|
|2017|   11|  1|  10|         15|    0.0|          34| 201711110|
|2017|   12| 12|  16|         12|    0.0|          47|2017121216|
|2017|   12| 24|  17|         11|    1.0|          50|2017122417|
|2017|    5|  7|  14|         21|    0.0|          70|  20175714|
|2018|    1| 18|  17|         10|    2.0|          45| 201811817|
|2018|   11|  1|  11|         11|    0.1|          65| 201811111|
|2018|    5| 20|  21|         20|    0.0|          59| 201852021|
+----+-----+---+----+-----------+-------+------------+----------+



#### Retrieve all data in the last six months of each year:
Filter: month > 6

In [37]:
readDF = spark.read.format("io.iguaz.v3io.spark.sql.kv").load(table_path) \
    .filter("month > 6").show()

+----+-----+---+----+-----------+-------+------------+----------+
|year|month|day|hour|degrees_cel|rain_ml|humidity_per|      time|
+----+-----+---+----+-----------+-------+------------+----------+
|2016|   12| 24|   9|         10|    0.1|          47| 201612249|
|2016|    7| 24|   7|         19|    0.0|          52|  20167247|
|2017|   11|  1|  10|         15|    0.0|          34| 201711110|
|2017|   12| 12|  16|         12|    0.0|          47|2017121216|
|2017|   12| 24|  17|         11|    1.0|          50|2017122417|
|2018|   11|  1|  11|         11|    0.1|          65| 201811111|
+----+-----+---+----+-----------+-------+------------+----------+



#### Retrieve all hours in Dec 24 of each year:
Filter: month == 12 AND day == 24 

In [39]:
readDF = spark.read.format("io.iguaz.v3io.spark.sql.kv").load(table_path) \
    .filter("month == 12 AND day == 24") \
    .show()

+----+-----+---+----+-----------+-------+------------+----------+
|year|month|day|hour|degrees_cel|rain_ml|humidity_per|      time|
+----+-----+---+----+-----------+-------+------------+----------+
|2016|   12| 24|   9|         10|    0.1|          47| 201612249|
|2017|   12| 24|  17|         11|    1.0|          50|2017122417|
+----+-----+---+----+-----------+-------+------------+----------+



#### Retrieve data in the last six months of each year between 08:00–20:00 in a day

In [40]:
readDF = spark.read.format("io.iguaz.v3io.spark.sql.kv").load(table_path) \
    .filter("month < 7 AND hour >= 8 AND hour <= 20") \
    .show()

+----+-----+---+----+-----------+-------+------------+---------+
|year|month|day|hour|degrees_cel|rain_ml|humidity_per|     time|
+----+-----+---+----+-----------+-------+------------+---------+
|2016|    3| 25|  17|         18|    0.2|          62|201632517|
|2017|    5|  7|  14|         21|    0.0|          70| 20175714|
|2018|    1| 18|  17|         10|    2.0|          45|201811817|
+----+-----+---+----+-----------+-------+------------+---------+



# Conditional Update on Iguazio

This example demonstrates how to conditionally update NoSQL table items by using the condition write option.  Each write call in the example is followed by matching read and show calls to read and display the value of the updated item in the target table after the write operation.

## Generate Data

The first write command writes an item (row) to a “cars” table . The item’s reg_license primary-key (identity-column) attribute is set to 7843321, the mode attribute is set to “Honda”, and the odometer attribute is set to `29321`.  The overwrite save mode is used to overwrite the table if it already exists and create it otherwise.  Reading the item from the table produces this output:

In [47]:
writeDF = spark.createDataFrame([("7843321", "Honda", 29321)],
                                ["reg_license", "model", "odometer"])

writeDF.write.format("io.iguaz.v3io.spark.sql.kv") \
    .option("key", "reg_license") \
    .mode("overwrite") \
    .save(os.path.join(os.getenv('V3IO_HOME'))+'/cars/')

readDF = spark.read.format("io.iguaz.v3io.spark.sql.kv") \
    .load(os.path.join(os.getenv('V3IO_HOME'))+'/cars/') \
    .show()

+-----------+-----+--------+
|reg_license|model|odometer|
+-----------+-----+--------+
|    7843321|Honda|   29321|
+-----------+-----+--------+



## Update on Condition

Update the odometer to `31718` on the condition of the new odometer value is greater than the old one's. This is to catch up the up-to-date value of odometer.

In [48]:
writeDF = spark.createDataFrame([("7843321", "Honda", 31718)],
                                ["reg_license", "model", "odometer"])

writeDF.write.format("io.iguaz.v3io.spark.sql.kv") \
    .option("key", "reg_license") \
    .option("condition", "${odometer} > odometer") \
    .mode("append") \
    .save(os.path.join(os.getenv('V3IO_HOME'))+'/cars/')

readDF = spark.read.format("io.iguaz.v3io.spark.sql.kv") \
    .load(os.path.join(os.getenv('V3IO_HOME'))+'/cars/') \
    .show()

+-----------+-----+--------+
|reg_license|model|odometer|
+-----------+-----+--------+
|    7843321|Honda|   31718|
+-----------+-----+--------+



**Congratulations** - you complete Sparl SQL and Anytics in a notebook on Iguazio.

# Clean Up

Prior to exiting, let's do housekeep to release disk space, computation and memory resources taken by this session.

## Remove Data
When you are done - cleaning the directory will be done by running the following command:

In [50]:
# Unmark the comment
# !rm -rf $HOME/PATH/*

## Stop Spark Session
In order to release compute and memory resources taken by Spark we recommend running the following command to stop the Spark session.

In [51]:
spark.stop()